In [1]:
import random

from collections import Iterable
from copy import deepcopy
from itertools import permutations, product
from queue import Queue

from IPython.display import HTML, display, clear_output

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
class Point(complex):
    "A point in the (x, y) plane."
    def __repr__(self): return '({}, {})'.format(self.x, self.y)
    x = property(lambda p: p.real)
    y = property(lambda p: p.imag)

Position = Point
Dim2 = Point

In [3]:
def argmax(inputs, f):
    return max(inputs, key=f)

In [4]:
class Board:
    COLORS = 'red lightgrey yellow slateblue plum chartreuse cyan coral olive lightgrey wheat'.split()
    
    def __init__(self, dimensions, colors=4, seed=42):
        self.seed = seed
        self.dimensions = dimensions
        self.colors = colors
        self.cells = dict()
        
        self._populate()
        self.original_cells = dict(self.cells)

    def _populate(self):
        random.seed(self.seed)
        for y in range(int(self.dimensions.y)):
            for x in range(int(self.dimensions.x)):
                self._set(x, y, random.randint(1, self.colors))
    
    def _get(self, x, y):
        return self.cells[Position(x, y)]
    
    def _set(self, x, y, color):
        self.cells[Position(x, y)] = color
        
    def _within_bounds(self, position):
        return (position.x >= 0 and position.y >= 0 and
                position.x < self.dimensions.x and position.y < self.dimensions.y)
                
    def show(self, in_html=False):
        cat = ''.join

        if in_html:
            html_table = '<table>{}</table>'
            html_row = '<tr>{}</tr>'
            html_td = '<td style="background-color:{}">{}</td>'
            
            table = list()
            for y in range(int(self.dimensions.y)):
                row = list()
                for x in range(int(self.dimensions.x)):
                    color = self._get(x, y)
                    row.append(html_td.format(Board.COLORS[color-1], color))
                table.append(html_row.format(cat(row)))
            
            display(HTML(html_table.format(cat(table))))

        else:
            for y in range(int(self.dimensions.y)):
                for x in range(int(self.dimensions.x)):
                    print(self._get(x, y), end=' ')
                print()
                
    def get_flood_size(self):
        count = 0
        q = Queue()
        seen = set()
        
        q.put(Position(0, 0))
        seen.add(Position(0, 0))
        flood_color = self._get(0, 0)

        while not q.empty():
            count += 1
            cell = q.get()

            neighbors = [Position(cell.x + 1, cell.y), # right
                         Position(cell.x, cell.y + 1), # down
                         Position(cell.x - 1, cell.y), # left
                         Position(cell.x, cell.y - 1)] # up

            for n_cell in neighbors:
                if self._within_bounds(n_cell) and n_cell not in seen:
                    n_color = self._get(n_cell.x, n_cell.y)
                    if n_color == flood_color:
                        q.put(n_cell)
                        seen.add(n_cell)

        return count

    def flood(self, colors):
        """Flood a single or a list of colors to the board.
        
        Return a list of tuples: (color, flood_size)
        """

        if not isinstance(colors, Iterable):
            colors = [colors]

        ret_list = list()

        for flood_color in colors:
            # stop if done
            if self.is_done():
                return ret_list

            # new count for every flood_color
            count = 0
            q = Queue()
            seen = set()

            q.put(Position(0, 0))
            seen.add(Position(0, 0))
            old_color = self._get(0, 0)

            while not q.empty():
                cell = q.get()
                cell_color = self._get(cell.x, cell.y)
                
                neighbors = [Position(cell.x + 1, cell.y), # right
                             Position(cell.x, cell.y + 1), # down
                             Position(cell.x - 1, cell.y), # left
                             Position(cell.x, cell.y - 1)] # up

                for n_cell in neighbors:
                    if self._within_bounds(n_cell) and n_cell not in seen:
                        n_color = self._get(n_cell.x, n_cell.y)
                        if cell_color == old_color:
                            if n_color == old_color or n_color == flood_color:
                                q.put(n_cell)
                                seen.add(n_cell)
                        elif cell_color == flood_color:
                            if n_color == flood_color:
                                q.put(n_cell)
                                seen.add(n_cell)
                
                # set the color of the current cell
                self._set(cell.x, cell.y, flood_color)
                count += 1
            
            ret_list.append((flood_color, count))

        return ret_list
    
    def potential_flood(self, colors):
        saved_cells = dict(self.cells)
        last_color, last_flood = self.flood(colors)[-1]
        self.reset(to_state=saved_cells)
        return last_flood
   
    def reset(self, to_state=None):
        self.cells.clear()
        if to_state:
            self.cells.update(to_state)
        else:
            self.cells.update(self.original_cells)
        
    def is_done(self):
        return len(set(self.cells.values())) == 1

In [5]:
board = Board(Dim2(12, 12))
board.show(True)

1,1,3,2,2,2,1,1,4,1,1,1
2,2,1,2,4,2,4,3,1,2,4,3
3,2,2,3,1,1,4,1,3,3,3,1
4,1,4,1,3,3,2,1,1,2,3,1
2,1,4,3,4,3,2,3,3,2,3,1
2,2,2,4,4,3,2,3,1,2,1,3
4,3,1,2,3,2,4,4,4,2,3,2
2,3,4,4,3,2,2,4,1,1,1,2
2,4,1,4,4,4,3,1,1,3,3,1
3,4,2,4,1,3,2,1,3,2,2,3
2,1,3,4,1,1,3,3,2,1,2,1


In [6]:
def naive(board, debug=False):
    board.reset()
    steps = list()
    prev_flood = 0
    while not board.is_done():
        color = random.randint(1, board.colors)
        _, flood = board.flood(color)[0]
        if flood != prev_flood:
            steps.append(color)
            prev_flood = flood
    return steps

steps = naive(board)
print('Naive method \n{} steps: {}'.format(len(steps), steps))

Naive method 
20 steps: [4, 3, 1, 2, 1, 3, 1, 2, 1, 2, 1, 3, 1, 2, 3, 4, 2, 1, 4, 3]


In [7]:
def immediate_maximum(board, debug=False):
    board.reset()
    steps = list()
    prev_flood = 0
    while not board.is_done():
        color = argmax(range(1, board.colors + 1), board.potential_flood)
        _, flood = board.flood(color)[0]
        if flood == prev_flood:
            print('Argmax did not work: color %d' % color)
            prev_flood = flood
        steps.append(color)
    return steps

steps = immediate_maximum(board)
print('Immediate Maximum method \n{} steps: {}'.format(len(steps), steps))

Immediate Maximum method 
14 steps: [2, 1, 2, 4, 3, 1, 2, 4, 1, 3, 2, 1, 4, 3]


## Comparing Naive to Immediate Maximum

In [8]:
diffs = list()
baddies = list()
while len(baddies) < 10:
    board = Board(Dim2(12, 12), seed=None)
    nai = naive(board)
    imm_max = immediate_maximum(board)
    diff = len(nai) - len(imm_max)
    if diff <= 1:
        board.reset()
        baddies.append((board, nai, imm_max))
        print('Found %d baddies' % len(baddies))
    diffs.append(diff)

Found 1 baddies
Found 2 baddies
Found 3 baddies
Found 4 baddies
Found 5 baddies
Found 6 baddies
Found 7 baddies
Found 8 baddies
Found 9 baddies
Found 10 baddies


In [9]:
diffs = np.array(diffs)
print('Count: %d' % len(diffs))
print('Average: %d' % np.average(diffs))
print('Min: %d' % np.min(diffs))
print('Max: %d' % np.max(diffs))
for idx, item in enumerate(baddies):
    board, nai, imm_max = item
    print('Board %d' % idx)
    print('Naive: {}'.format(nai))
    print('Immediate Max: {}'.format(imm_max))
    print('Diff: %d' % (len(nai) - len(imm_max)))
    print()

Count: 140
Average: 4
Min: -3
Max: 10
Board 0
Naive: [3, 2, 1, 2, 4, 2, 4, 3, 1, 4, 2, 4, 2, 1, 4, 3, 2, 1]
Immediate Max: [2, 4, 1, 3, 4, 2, 1, 3, 4, 1, 2, 3, 4, 1, 2, 4, 3, 1, 2]
Diff: -1

Board 1
Naive: [3, 1, 2, 1, 2, 1, 4, 2, 1, 4, 2, 3, 1, 4]
Immediate Max: [2, 1, 2, 3, 4, 2, 1, 4, 3, 1, 2, 4, 3]
Diff: 1

Board 2
Naive: [4, 1, 4, 2, 1, 3, 2, 1, 2, 4, 3, 2, 1, 3, 2]
Immediate Max: [2, 1, 4, 2, 1, 4, 3, 2, 1, 4, 3, 1, 2, 4]
Diff: 1

Board 3
Naive: [4, 1, 3, 2, 4, 3, 4, 1, 2, 4, 2, 3, 1, 4, 3, 2, 1]
Immediate Max: [1, 3, 4, 1, 3, 2, 4, 1, 3, 4, 2, 1, 3, 4, 2, 1]
Diff: 1

Board 4
Naive: [4, 2, 1, 2, 4, 1, 2, 4, 3, 1, 4, 3, 2]
Immediate Max: [2, 3, 1, 2, 4, 1, 2, 4, 3, 1, 2, 3, 4]
Diff: 0

Board 5
Naive: [4, 3, 4, 2, 3, 1, 4, 1, 2, 4, 3, 4, 2, 1, 2, 3, 2, 4]
Immediate Max: [3, 4, 3, 4, 2, 1, 4, 2, 3, 1, 4, 2, 3, 1, 4, 2, 3]
Diff: 1

Board 6
Naive: [3, 4, 2, 4, 1, 2, 3, 4, 2, 3, 1, 4, 2, 4, 3, 1]
Immediate Max: [4, 2, 1, 4, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4]
Diff: 1

Board 7
Naive: [4, 3

In [25]:
def layered_maximum(board, layers, debug=False):
    board.reset()
    board_colors = range(1, board.colors + 1)
    perms = product(board_colors, repeat=layers)
    
    # filter perms which have adjacent duplicated
    def p_filter(tup):
        for idx in range(1, len(tup)):
            if tup[idx] == tup[idx-1]:
                return False
        return True

    perms = list(filter(p_filter, perms))
    if debug:
        print('Possible perms: {}'.format(perms))

    steps = list()
    while not board.is_done():
        best_colors = argmax(perms, board.potential_flood)
        ret_list = board.flood(best_colors)
        colors = [col for col, size in ret_list]
        steps.extend(colors)
    return steps

steps = layered_maximum(board, 2, debug=True)
print('Layered Maximum (2 layers) method \n{} steps: {}'.format(len(steps), steps))

Possible perms: [(1, 2), (1, 3), (1, 4), (2, 1), (2, 3), (2, 4), (3, 1), (3, 2), (3, 4), (4, 1), (4, 2), (4, 3)]
Layered Maximum (2 layers) method 
14 steps: [1, 2, 3, 1, 2, 4, 1, 2, 4, 3, 1, 2, 1, 4]


In [11]:
steps = layered_maximum(board, 3, debug=True)
print('Layered Maximum (3 layers) method \n{} steps: {}'.format(len(steps), steps))

Possible perms: [(1, 2, 1), (1, 2, 3), (1, 2, 4), (1, 3, 1), (1, 3, 2), (1, 3, 4), (1, 4, 1), (1, 4, 2), (1, 4, 3), (2, 1, 2), (2, 1, 3), (2, 1, 4), (2, 3, 1), (2, 3, 2), (2, 3, 4), (2, 4, 1), (2, 4, 2), (2, 4, 3), (3, 1, 2), (3, 1, 3), (3, 1, 4), (3, 2, 1), (3, 2, 3), (3, 2, 4), (3, 4, 1), (3, 4, 2), (3, 4, 3), (4, 1, 2), (4, 1, 3), (4, 1, 4), (4, 2, 1), (4, 2, 3), (4, 2, 4), (4, 3, 1), (4, 3, 2), (4, 3, 4)]
Layered Maximum (3 layers) method 
12 steps: [1, 2, 3, 4, 1, 2, 4, 1, 3, 2, 1, 4]


In [23]:
steps = layered_maximum(board, 4)
print('Layered Maximum (4 layers) method \n{} steps: {}'.format(len(steps), steps))

Layered Maximum (4 layers) method 
16 steps: [1, 2, 3, 1, 2, 4, 1, 2, 3, 4, 1, 3, 1, 2, 1, 2]


## Comparing Layered Maximum to previous methods

In [ ]:
runs = 100

runs_naive = list()
runs_imm = list()
runs_lay2 = list()
runs_lay3 = list()
runs_lay4 = list()

for run in range(runs):
    board = Board(Dim2(12, 12), seed=None)
    runs_naive.append(naive(board))
    runs_imm.append(immediate_maximum(board))
    runs_lay2.append(layered_maximum(board, 2))
    runs_lay3.append(layered_maximum(board, 3))
    runs_lay4.append(layered_maximum(board, 4))
    
    if run % 20 == 0:
        print('Finished %d...' % run)

In [ ]:
# plt.plot(np.linspace(1, runs, runs), list(map(len, runs_naive)), label='naive')
plt.plot(np.linspace(1, runs, runs), list(map(len, runs_imm)), label='imm max')
plt.plot(np.linspace(1, runs, runs), list(map(len, runs_lay2)), label='2 layered max')
plt.plot(np.linspace(1, runs, runs), list(map(len, runs_lay3)), label='3 layered max')
plt.plot(np.linspace(1, runs, runs), list(map(len, runs_lay4)), label='4 layered max')

plt.title('Solving methods compared')
plt.xlabel('Run #')
plt.ylabel('Steps')
plt.legend()

# plt.savefig('test2.png', dpi=200)
plt.show()

## Tests

In [20]:
board = Board(Dim2(12, 12), seed=None)
board.show(True)

3,1,1,2,1,1,2,1,3,4,3,3
3,1,2,2,2,1,1,4,3,3,1,4
1,2,3,1,3,2,2,1,3,4,1,3
3,2,3,2,1,4,2,2,4,4,3,4
1,3,3,2,4,1,1,3,2,2,4,1
2,2,3,3,1,3,2,3,4,2,4,3
3,4,1,4,3,2,1,4,1,3,3,4
3,2,2,2,3,2,1,2,4,4,2,1
2,1,4,1,2,1,2,2,4,4,3,3
3,1,4,4,4,1,1,1,3,4,1,3
4,2,1,1,2,4,4,1,2,2,3,2


In [38]:
board.reset()
board.flood([1, 2, 3, 1, 2, 4, 1, 2, 3, 4, 1, 3])
board.flood([1, 2])

[(1, 141), (2, 144)]

In [26]:
print('Naive:       {}'.format(naive(board)))
print('Imm Max:     {}'.format(immediate_maximum(board)))
print('2 Layer Max: {}'.format(layered_maximum(board, 2)))
print('3 Layer Max: {}'.format(layered_maximum(board, 3)))
print('4 Layer Max: {}'.format(layered_maximum(board, 4)))

Naive:       [2, 1, 2, 3, 2, 4, 3, 2, 1, 2, 3, 2, 1, 4, 2, 3, 1]
Imm Max:     [1, 2, 3, 1, 2, 4, 1, 2, 3, 4, 1, 3, 2]
2 Layer Max: [1, 2, 3, 1, 2, 4, 1, 2, 4, 3, 1, 2, 1, 4]
3 Layer Max: [1, 2, 3, 1, 2, 4, 1, 2, 3, 4, 1, 3, 1, 2]
4 Layer Max: [1, 2, 3, 1, 2, 4, 1, 2, 3, 4, 1, 3, 1, 2]


In [ ]:
_colors = list(range(1, board.colors + 1))
list(chain(_colors, permutations(_colors, 2), permutations(_colors, 3)))